## Import package

In [1]:
import pandas as pd
import os
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import gc

pd.options.mode.chained_assignment = None

In [2]:
preprocess_path = '../data/mimiciii/preprocess_data/'
temporal_path = '../data/mimiciii/temporal_dataset/'

# Features

In [3]:
# last row don't understand meaning
features = ['Shock_Index', 'elixhauser', 'SIRS', 'gender', 're_admission', 'GCS', 'SOFA', 'age', 'Albumin',
           'Arterial_pH', 'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'SGPT', 
           'Arterial_blood_gas', 'BUN', 'HCO3', 'INR', 'Arterial_lactate', 'CO2', 'Creatinine', 
           'Ionised_Ca', 'PT', 'Platelets_count', 'SGOT', 'Total_bili', 'WBC_count', 'DiaBP', 'SysBP', 
           'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 'Weight_kg', 'HR', 'SpO2'
           'input_total_tev', 'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc', 'mechvent']

binary_features = ['gender', 're_admission', 'mechvent']
norm_features = ['Shock_Index', 'elixhauser', 'SIRS', 'GCS', 'SOFA', 'age', 'Albumin', 'Arterial_pH', 
                'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'Arterial_blood_gas', 
                'HCO3', 'Arterial_lactate', 'CO2', 'Ionised_Ca', 'PT', 'Platelets_count', 'WBC_count', 
                'DiaBP', 'SysBP', 'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 
                'Weight_kg', 'HR', ]
log_features = ['SGPT', 'BUN', 'INR', 'Creatinine', 'SGOT', 'Total_bili', 'SpO2', 'input_total_tev',
                'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc']

## Load require data

In [4]:
CHARTEVENTS1 = pd.read_csv(preprocess_path + 'CHARTEVENTS1.csv')
CHARTEVENTS2 = pd.read_csv(preprocess_path + 'CHARTEVENTS2.csv')
CHARTEVENTS3 = pd.read_csv(preprocess_path + 'CHARTEVENTS3.csv')
CHARTEVENTS4 = pd.read_csv(preprocess_path + 'CHARTEVENTS4.csv')
CHARTEVENTS5 = pd.read_csv(preprocess_path + 'CHARTEVENTS5.csv')
CHARTEVENTS6 = pd.read_csv(preprocess_path + 'CHARTEVENTS6.csv')
CHARTEVENTS7 = pd.read_csv(preprocess_path + 'CHARTEVENTS7.csv')
CHARTEVENTS8 = pd.read_csv(preprocess_path + 'CHARTEVENTS8.csv')
CHARTEVENTS9 = pd.read_csv(preprocess_path + 'CHARTEVENTS9.csv')
LABEVENTS = pd.read_csv(preprocess_path + 'LABEVENTS.csv')


datas = [CHARTEVENTS1, CHARTEVENTS2, CHARTEVENTS3, CHARTEVENTS4, CHARTEVENTS5, CHARTEVENTS6, CHARTEVENTS7, 
         CHARTEVENTS8, CHARTEVENTS9, LABEVENTS]

for i in tqdm(range(len(datas))):
    for column in datas[i].columns:
        if 'TIME' in column or 'DATE' in column or 'DOB' == column or 'DOD' == column or 'DOD_HOSP' == column or 'DOD_SSN' == column:
            datas[i][column] = datas[i][column].apply(lambda x : pd.Timestamp(x))

/home/toooot/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/toooot/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/toooot/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,10,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [02:54<00:00, 17.41s/it]


## unified unit

### temperature (celcius)

In [35]:
def changeTempUnit(series):
    if series['ITEMID'] in [678, 679, 223761]:
        try:
            series['VALUENUM'] = (series['VALUENUM'] - 32) * 5 / 9
        except:
            series['VALUENUM'] = np.nan

for i in tqdm(range(len(datas))):
    datas[i]['VALUENUM'] = datas[i].apply(changeTempUnit, axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [07:18<00:00, 39.86s/it]


### FiO2 (torr)

## Load variable range

In [26]:
var_range = pd.read_csv('../data/mimiciii/variable_range.csv')
var_range.index = var_range.iloc[:, 0]
var_range = var_range.iloc[:, 1:]
var_range['GROUP_ID'] = var_range['GROUP_ID'].apply(lambda x : [int(s) for s in x.split(',')] if type(x) == type('') else x)
var_range

,OUTLIER LOW,VALID LOW,IMPUTE,VALID HIGH,OUTLIER HIGH,GROUP_ID
Unnamed: 0,,,,,,
GCS,0.0,0.00,NaN,15.0,15.0,[198]
SOFA,0.0,0.00,NaN,24.0,24.0,[227428]
Albumin,0.0,0.60,3.10,6.0,60.0,"[50862, 772, 1521, 227456]"
Arterial pH,NaN,NaN,NaN,NaN,NaN,"[780, 1126, 223830, 50820]"
Calcium,NaN,NaN,NaN,NaN,NaN,"[225625, 50893, 786, 1522]"
Glucose,0.0,33.00,128.00,2000.0,2200.0,"[220621, 225664, 50809, 811, 807, 50931, 22653..."
Hemoglobin,0.0,0.00,10.20,25.0,30.0,"[220228, 51222, 50811, 814]"
Magnesium,0.0,0.00,2.00,20.0,22.0,"[220635, 50960, 1532, 821]"
PTT,0.0,18.80,34.30,150.0,150.0,"[227466, 825, 1533, 51275]"


## Merge features to dataset

In [33]:
def merge(dataset, feature, EVENTS, var_range):
    # find all feature records in EVENTS
    event = EVENTS[[id in var_range.loc[feature]['GROUP_ID'] for id in EVENTS['ITEMID']]]

    for i in range(len(event)):
        
        subject_id = event.iloc[i]['SUBJECT_ID']
        hadm_id = event.iloc[i]['HADM_ID']
        time = event.iloc[i]['CHARTTIME']
        
        # detect outlier value, if out of outlier range then set to missing value
        value = event.iloc[i]['VALUENUM']
        if var_range.loc[feature].notnull()[0]:
            if value > var_range.loc[feature]['OUTLIER HIGH']:
                value = np.nan
            elif value > var_range.loc[feature]['VALID HIGH']:
                value = var_range.loc[feature]['VALID HIGH']
            elif value < var_range.loc[feature]['OUTLIER LOW']:
                value = np.nan
            elif value < var_range.loc[feature]['VALID LOW']:
                value = var_range.loc[feature]['VALID LOW']
        
        index = dataset.query(f'SUBJECT_ID == {subject_id} & HADM_ID == {hadm_id}').index
        
        for idx in index:
            if time - dataset.iloc[idx]['STARTTIME'] >= pd.Timedelta('0') and time - dataset.iloc[idx]['ENDTIME'] <= pd.Timedelta('0'):
                if dataset['records'][idx] == 0:
                    dataset[feature][idx] = value
                else:
                    record = dataset['records'][idx]
                    # use mean when multiple records in same period
                    dataset[feature][idx] = value / (record + 1) + record / (record + 1) * dataset[feature][idx]
                dataset['records'][idx] += 1
                break
                    
for hour in tqdm(range(4, 0, -1)):
    path = temporal_path + f'dataset_split_{hour}_hour.csv'

    dataset = pd.read_csv(path)

    # change time column data's type to timestamp type
    dataset['STARTTIME'] = dataset['STARTTIME'].apply(lambda x : pd.Timestamp(x))
    dataset['ENDTIME'] = dataset['ENDTIME'].apply(lambda x : pd.Timestamp(x))
    dataset['DEATHTIME'] = dataset['DEATHTIME'].apply(lambda x : pd.Timestamp(x))
    dataset = dataset.loc[:, ['SUBJECT_ID', 'HADM_ID', 'STARTTIME', 'ENDTIME', 'Gender', 
                              'Age', 'DEATHTIME', 're_admission']]
    # dataset.head()                    


    for feature in tqdm(var_range.index):
        # count records in same period
        dataset['records'] = pd.Series([0] * len(dataset), dtype = np.int64)

        # add new feature column
        dataset[feature] = pd.Series([np.nan] * len(dataset), dtype = np.float64)

        # add data to new feature column with corresponding subject_id, hadm_id, icustay_id and time
        merge(dataset, feature, LABEVENTS, var_range)
        merge(dataset, feature, CHARTEVENTS1, var_range)
        merge(dataset, feature, CHARTEVENTS2, var_range)
        merge(dataset, feature, CHARTEVENTS3, var_range)
        merge(dataset, feature, CHARTEVENTS4, var_range)
        merge(dataset, feature, CHARTEVENTS5, var_range)
        merge(dataset, feature, CHARTEVENTS6, var_range)
        merge(dataset, feature, CHARTEVENTS7, var_range)
        merge(dataset, feature, CHARTEVENTS8, var_range)
        merge(dataset, feature, CHARTEVENTS9, var_range)

    dataset.to_csv(temporal_path + f'dataset_split_{hour}_hour_after_merge.csv', index=False)

 25%|███████████████████████████████████████▎                                                                                                                     | 1/4 [76:24:52<229:14:38, 275092.89s/it]


KeyboardInterrupt: 

## Load dataset split hour after merge and require data

In [7]:
OUTPUTEVENTS = pd.read_csv(dest_path + 'OUTPUTEVENTS.csv')
INPUTEVENTS_MV = pd.read_csv(dest_path + 'INPUTEVENTS_MV.csv')
INPUTEVENTS_CV = pd.read_csv(dest_path + 'INPUTEVENTS_CV.csv')
dataset = pd.read_csv(temporal_path + 'dataset_split_4_hour_after_merge.csv')
dataset = dataset.iloc[:, 1:]
dataset.head()

,SUBJECT_ID,HADM_ID,STARTTIME,ENDTIME,Gender,Age,DEATHTIME,re_admission,records,GCS,...,Systolic Blood Pressure,Mean Blood Pressure,PaCO2,PaO2,FiO2,Respiratory rate,Temperature (Celsius),Weight (kg),Heart Rate,SpO2
0,21,111970,2135-01-30 20:50:00,2135-01-31 00:50:00,1,87.882192,2135-02-08 02:08:00,0.0,3,14.0,...,NaN,66.333333,NaN,NaN,NaN,14.666667,NaN,64.0,74.000000,99.333333
1,21,111970,2135-01-31 00:50:00,2135-01-31 04:50:00,1,87.884932,2135-02-08 02:08:00,0.0,4,NaN,...,NaN,62.444434,NaN,NaN,NaN,16.500000,NaN,NaN,75.750000,96.500000
2,21,111970,2135-01-31 04:50:00,2135-01-31 08:50:00,1,87.884932,2135-02-08 02:08:00,0.0,5,13.0,...,NaN,56.333340,NaN,NaN,NaN,16.500000,NaN,NaN,76.200000,98.000000
3,21,111970,2135-01-31 08:50:00,2135-01-31 12:50:00,1,87.884932,2135-02-08 02:08:00,0.0,6,14.0,...,NaN,61.750000,NaN,NaN,NaN,16.875000,NaN,NaN,77.714286,92.666667
4,21,111970,2135-01-31 12:50:00,2135-01-31 16:50:00,1,87.884932,2135-02-08 02:08:00,0.0,12,14.0,...,NaN,51.633330,NaN,NaN,NaN,17.583333,NaN,NaN,72.083333,96.416667


In [17]:
dataset.columns

Index(['SUBJECT_ID', 'HADM_ID', 'STARTTIME', 'ENDTIME', 'Gender', 'Age',
       'DEATHTIME', 're_admission', 'records', 'GCS', 'SOFA', 'Albumin',
       'Arterial pH', 'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT',
       'Postassium', 'SGPT', 'BUN', 'HCO3', 'INR', 'Arterial Lactate', 'CO2',
       'Creatinine', 'Ionized Calcium', 'PT', 'Platelets Count', 'SGOT',
       'Total Bilirubin', 'White Blood Cell Count', 'Diastolic Blood Pressure',
       'Systolic Blood Pressure', 'Mean Blood Pressure', 'PaCO2', 'PaO2',
       'FiO2', 'Respiratory rate', 'Temperature (Celsius)', 'Weight (kg)',
       'Heart Rate', 'SpO2', 'Shock Index'],
      dtype='object')

### shock index = HR / SBP (Heart rate / systolic blood pressure)

In [11]:
def get_shock_index(series):
    try:
        return series['Heart Rate'] / series['Systolic Blood Pressure']
    except:
        return np.nan

dataset['Shock Index'] = dataset.apply(get_shock_index, axis=1)

### elixhauser

### SIRS (Use diagnosis? or use criteria ?)

SIRS Criteria (>= 2 meets SIRS definition)
1. Temp > $38^{o}C$ ($100.4^{o}F$) or < $36^{o}C$ ($96.8^{o}F$)
2. Heart rate > 90
3. Respiratory rate > 20 or $PaCO_{2}$ < 32 mm Hg
4. WBC > 12,000/$mm^{3}$, < 4,000/$mm^{3}$, or > 10% bands (?)

In [ ]:
def get_SIRS(series):
    sum = 0
    if series['Temperature (Celsius)'] > 38 or series['Temperature (Celsius)'] < 36:
        sum += 1
    if series['Heart Rate'] > 90:
        sum += 1
    if series['Respiratory rate'] > 20 or series['PaCO2'] < 32:
        sum += 1
    if series['White Blood Cell Count'] > 12000 or series['White Blood Cell Count'] < 4000:
        sum += 1
    return sum >= 2
    
dataset['SIRS'] = dataset.apply(get_SIRS, axis=1)

### SOFA

| System                                       | 0           |
| -----------                                  | ----------- |
| $PaO_{2}/FiO_{2}$ mmHg                       | Title       |
| Platelets, $x10^{3}/ \mu L$                  | Text        |
| Glasgow Coma Scale                           |             |
| Bilirubin, mm/dL ($\mu$mo;/L)                |             |
| Mean arterial pressure OR administration of vasoactive agents required                                       |             |
| Creatinine, mg/dL (μmol/L) (or urine output) |             |

In [ ]:
def get_SOFA(series):
    score = 0
    # PaO2/FiO2 mmHg
    if series['PaO2/FiO2 ratio'] < 100 and series['Mech Vent']:
        score += 4
    elif series['PaO2/FiO2 ratio'] < 200 and series['Mech Vent']:
        score += 3
    elif series['PaO2/FiO2 ratio'] < 300:
        score += 2
    elif series['PaO2/FiO2 ratio'] < 400:
        score += 1
    
    # Platelets, x10^3/μL
    if series['Platelets Count'] < 20:
        score += 4
    elif series['Platelets Count'] < 50:
        score += 3
    elif series['Platelets Count'] < 100:
        score += 2
    elif series['Platelets Count'] < 150:
        score += 1
        
    # GCS (Glasgow Coma Scale)
    if series['GCS'] < 6:
        score += 4
    elif series['GCS'] < 10:
        score += 3
    elif series['GCS'] < 13:
        score += 2
    elif series['GCS'] < 15:
        score += 1
    
    # bilirubin, mg/dL (μmol/L)
    if series['Total Bilirubin'] >= 12: # > 204
        score += 4
    elif series['Total Bilirubin'] >= 6: # >= 102
        score += 3
    elif series['Total Bilirubin'] >= 2: # >= 33
        score += 2
    elif series['Total Bilirubin'] >= 1.2: # >= 20
        score += 1
    
    # Mean arterial pressure OR administration of vasoactive agents required
    # get Dopamine and epinephrine

    
    # Creatinine, mg/dL (μmol/L) (or urine (mL/day) output)
    if series['Creatinine'] >= 5.0 or dataset['urine'] < 200:
        score += 4
    elif series['Creatinine'] >= 3.5 or dataset['urine'] < 500:
        score += 3
    elif series['Creatinine'] >= 2.0:
        score += 2
    elif series['Creatinine'] >= 1.2:
        score += 1
    
    
dataset['SOFA'] = dataset.apply(get_SOFA. axis=1)

### PaO2/FiO2 ratio

In [18]:
def get_PaO2_FiO2_ratio(series):
    try:
        return series['PaO2'] / series['FiO2']
    except:
        return np.nan

dataset['PaO2/FiO2 ratio'] = dataset.apply(get_PaO2_FiO2_ratio, axis=1)

### Output total

### Output four hour

### Input total

### Input four hour

### max_dose_vaso

### cumulated balance tev

### Mechanical Ventilation (in produreevents_mv)